## The first part of an LCEL chain must be a runnable type.

In [2]:
from langchain_core.prompts.chat import ChatPromptTemplate
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

bad_first_input = {
    "film_required_age": 18,
}

prompt = ChatPromptTemplate.from_template(
    "Generate a film title, the age is {film_required_age}"
)

# This will error:
bad_chain = bad_first_input | prompt

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'int'>

In [3]:
# All of these chains enforce the runnable interface:
first_good_input = {"film_required_age": itemgetter("film_required_age")}

# Creating a dictionary within a RunnableLambda:
second_good_input = RunnableLambda(lambda x: { "film_required_age": x["film_required_age"] } )

third_good_input = RunnablePassthrough()
fourth_good_input = {"film_required_age": RunnablePassthrough()}
# You can also create a chain starting with RunnableParallel

first_good_chain = first_good_input | prompt
second_good_chain = second_good_input | prompt
third_good_chain = third_good_input | prompt
fourth_good_chain = fourth_good_input | prompt

first_good_chain.invoke({
    "film_required_age": 18
})

# ...


ChatPromptValue(messages=[HumanMessage(content='Generate a film title, the age is 18')])

## Order Matters

In [4]:
bad_order_chain = prompt | first_good_input
bad_order_chain.invoke({"film_required_age": 18})

TypeError: 'ChatPromptValue' object is not subscriptable

Given that your story generation will require multiple sequential prompts you can use a create `SequentialChain` to chain multiple prompts together. The `SequentialChain` will need to re-use the outputs of the previous prompt and use it as the input for the next prompt.


In [5]:
from langchain_core.prompts.chat import ChatPromptTemplate

In [6]:
character_generation_prompt = ChatPromptTemplate.from_template(
    """I want you to brainstorm 3 - 5 characters for my short story. The genre is {genre}.
    Each character must have a Name and a Biography.
    You must provide a name and biography for each character, this is very important!
    ---
    Example response:
    Name: CharWiz, Biography: A wizard who is a master of magic.
    Name: CharWar, Biography: A warrior who is a master of the sword.
    ---
    Characters: """
)

plot_generation_prompt = ChatPromptTemplate.from_template(
    """Given the following characters and the genre, create an effective plot for a short story:
    Characters:
    {characters} 
    ---
    Genre: {genre}
    ---
    Plot: """
    )

scene_generation_plot_prompt = ChatPromptTemplate.from_template(
    """Act as an effective content creator. 
    Given multiple characters and a plot you are responsible generating the various scenes for each act. 
    
    You must de-compose the plot into multiple effective scenes:

    ---
    Characters:
    {characters}
    ---
    Genre: {genre}
    ---
    Plot: {plot}
    ---
    Example response:
    Scenes:
    Scene 1: Some text here.
    Scene 2: Some text here.
    Scene 3: Some text here.
    ----
    Scenes:
    """
)

---

## ItemGetter and RunnableLambda

In [7]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

In [8]:
chain = RunnablePassthrough() | {
    "genre": itemgetter("genre"),
}
chain.invoke({"genre": "fantasy"})

{'genre': 'fantasy'}

In [9]:
# Another way to achieve the same thing:
chain = RunnableLambda(lambda x: {"genre": x["genre"]}) | {"genre": itemgetter("genre")}
chain.invoke({"genre": "fantasy"})
# {'genre': 'fantasy'}

{'genre': 'fantasy'}

In [10]:
chain = RunnablePassthrough() | {
    "genre": itemgetter("genre"),
    "upper_case_genre": lambda x: x["genre"].upper(),
    "lower_case_genre": RunnableLambda(lambda x: x["genre"].lower()),
}
chain.invoke({"genre": "fantasy"})

{'genre': 'fantasy',
 'upper_case_genre': 'FANTASY',
 'lower_case_genre': 'fantasy'}

---------------------------------------------

## RunnableParallel

In [11]:
from langchain_core.runnables import RunnableParallel

master_chain = RunnablePassthrough() | {
    "genre": itemgetter("genre"),
    "upper_case_genre": lambda x: x["genre"].upper(),
    "lower_case_genre": RunnableLambda(lambda x: x["genre"].lower()),   
}

master_chain_two = RunnablePassthrough() | RunnableParallel(
        genre=itemgetter("genre"),
        upper_case_genre=lambda x: x["genre"].upper(),
        lower_case_genre=RunnableLambda(lambda x: x["genre"].lower()),
)

story_result = master_chain.invoke({"genre": "Fantasy"})
print(f"master chain: {story_result}")

story_result = master_chain_two.invoke({"genre": "Fantasy"})
print(f"master chain two: {story_result}")

master chain: {'genre': 'Fantasy', 'upper_case_genre': 'FANTASY', 'lower_case_genre': 'fantasy'}
master chain two: {'genre': 'Fantasy', 'upper_case_genre': 'FANTASY', 'lower_case_genre': 'fantasy'}


--------------------------------------------------------------------------------

In [14]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [15]:
# Create the chat model:
model = ChatOpenAI()

# Create the sub-chains:
character_generation_chain = character_generation_prompt | model |  StrOutputParser()
plot_generation_chain = plot_generation_prompt | model | StrOutputParser()                                                               
scene_generation_plot_chain = scene_generation_plot_prompt | model | StrOutputParser()

In [16]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from operator import itemgetter

master_chain = (
    {"characters": character_generation_chain, "genre": RunnablePassthrough()}
    | RunnableParallel(
        characters=itemgetter("characters"),
        genre=itemgetter("genre"),
        plot=plot_generation_chain,
    )
    | RunnableParallel(
        characters=itemgetter("characters"),
        genre=itemgetter("genre"),
        plot=itemgetter("plot"),
        scenes=scene_generation_plot_chain,
    )
)

story_result = master_chain.invoke({"genre": "Fantasy"})

In [17]:
print(story_result['scenes'])

Scene 1: Elara practices her magical abilities in a secluded clearing, determined to perfect her skills and prove herself to her family and the world.

Scene 2: Thorn sneaks through the shadows of a bustling marketplace, using his quick wit and agility to avoid detection as he gathers information on the dark sorcerer's whereabouts.

Scene 3: Seraphina trains with her legendary sword, honing her skills and preparing for the battle ahead as she vows to protect the innocent and uphold justice in the face of darkness.

Scene 4: Orion roams the forests, his bow at the ready as he communicates with the animals and senses the growing threat of the dark sorcerer, determined to protect the land from any harm.

Scene 5: The group comes together, forming a powerful team as they embark on their journey to confront the dark sorcerer, facing treacherous obstacles and mythical creatures along the way.

Scene 6: Lyra's hidden tower comes into view, shrouded in mystery and magic, as she emerges to offe

---


## Sequential Story Scene Generation:


In [18]:
# Extracting the scenes using .split('\n') and removing empty strings:
scenes = [scene for scene in story_result["scenes"].split("\n") if scene]
generated_scenes = []
previous_scene_summary = ""

In [19]:
character_script_prompt = ChatPromptTemplate.from_template(
    template="""Given the following characters: {characters} and the genre: {genre}, create an effective character script for a scene.

    You must follow the following principles:
    - Use the Previous Scene Summary: {previous_scene_summary} to avoid repeating yourself.
    - Use the Plot: {plot} to create an effective scene character script.
    - Currently you are generating the character dialogue script for the following scene: {scene}

    ---
    Here is an example response:
    SCENE 1: ANNA'S APARTMENT

    (ANNA is sorting through old books when there is a knock at the door. She opens it to reveal JOHN.)
    ANNA: Can I help you, sir?
    JOHN: Perhaps, I think it's me who can help you. I heard you're researching time travel.
    (Anna looks intrigued but also cautious.)
    ANNA: That's right, but how do you know?
    JOHN: You could say... I'm a primary source.

    ---
    SCENE NUMBER: {index}

    """,
)

summarize_prompt = ChatPromptTemplate.from_template(
    template="""Given a character script create a summary of the scene. Character script: {character_script}""",
)

------------------------------------------------------------

In [20]:
# Loading a chat model:
model = ChatOpenAI(model='gpt-3.5-turbo-16k')

# Create the LCEL chains:
character_script_generation_chain = (
    {
        "characters": RunnablePassthrough(),
        "genre": RunnablePassthrough(),
        "previous_scene_summary": RunnablePassthrough(),
        "plot": RunnablePassthrough(),
        "scene": RunnablePassthrough(),
        "index": RunnablePassthrough(),
    }
    | character_script_prompt
    | model
    | StrOutputParser()
)

summarize_chain = summarize_prompt | model | StrOutputParser()

# You might want to use tqdm here to track the progress, or use all of the scenes:
for index, scene in enumerate(scenes[0:5]):
    
    # # Create a scene generation:
    scene_result = character_script_generation_chain.invoke(
        {
            "characters": story_result["characters"],
            "genre": "fantasy",
            "previous_scene_summary": previous_scene_summary,
            "index": index,
        }
    )

    # Store the generated scenes:
    generated_scenes.append(
        {"character_script": scene_result, "scene": scenes[index]}
    )

    # If this is the first scene then we don't have a previous scene summary:
    if index == 0:
        previous_scene_summary = scene_result
    else:
        # If this is the second scene or greater then we can use and generate a summary:
        summary_result = summarize_chain.invoke(
            {"character_script": scene_result}
        )
        previous_scene_summary = summary_result

--------------------------------------------------------------------------------

In [26]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
import pandas as pd

In [27]:
df = pd.DataFrame(generated_scenes)

In [28]:
df.head()

,character_script,scene
0,"SCENE 1: THE ANCIENT FOREST\n\n(ELARA, THORN, ...",Scene 1: Elara practices her magical abilities...
1,"SCENE 2: THE ANCIENT TEMPLE\n\n(ELARA, THORN, ...",Scene 2: Thorn sneaks through the shadows of a...
2,SCENE 2: ENTRANCE OF THE ANCIENT TEMPLE\n\n(Th...,Scene 3: Seraphina trains with her legendary s...
3,SCENE 3: ANCIENT TEMPLE ENTRANCE\n\n(The group...,"Scene 4: Orion roams the forests, his bow at t..."
4,SCENE 4: ANCIENT TEMPLE ENTRANCE\n\n(The group...,"Scene 5: The group comes together, forming a p..."


In [31]:
df.shape

(5, 2)

In [32]:
all_character_script_text = "\n".join(df.character_script.tolist())

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500, chunk_overlap=200
)

docs = text_splitter.create_documents([all_character_script_text])

In [33]:
chain = load_summarize_chain(llm=model, chain_type="map_reduce")
summary = chain.invoke(docs)
print(summary['output_text'])

A group of adventurers gather in an ancient forest to prepare for a dangerous quest to retrieve a powerful artifact. They discuss potential dangers and obstacles, remaining unified and determined. They stand before the entrance of an ancient temple, sensing lurking dangers and a powerful presence. Despite their trepidation, they push open the doors and enter with excitement.
